# Handy Utils to do Vector Search on Collections

## Configuration

In [1]:
class MyConfig:
    pass
MY_CONFIG = MyConfig()

MY_CONFIG.EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
MY_CONFIG.EMBEDDING_LENGTH = 384

MY_CONFIG.DB_URI = './rag_1_dpk.db'  # For embedded instance
#MY_CONFIG.DB_URI = 'http://localhost:19530'  # For Docker instance
MY_CONFIG.COLLECTION_NAME = 'dpk_walmart_docs'

## Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, close other notebooks that may be accessing the db, and try again</span>


In [2]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_1_dpk.db


## Setup Embeddings

Two choices here. 

1. use sentence transformers directly
2. use Milvus model wrapper

In [3]:
## Option 1 - use sentence transformers directly

# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(MY_CONFIG.EMBEDDING_MODEL)

def get_embeddings (str):
    embeddings = embedding_model.encode(str, normalize_embeddings=True)
    return embeddings

/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee/examples/notebooks/rag/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee/examples/notebooks/rag/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
## Option 2 - Milvus model
from pymilvus import model

# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'


# embedding_fn = model.DefaultEmbeddingFunction()

## initialize the SentenceTransformerEmbeddingFunction
embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name = MY_CONFIG.EMBEDDING_MODEL,
    device='cpu' # this will work on all devices (KIS)
)

In [5]:
# Test Embeddings
text = 'Paris 2024 Olympics'
embeddings = get_embeddings(text)
print ('sentence transformer : embeddings len =', len(embeddings))
print ('sentence transformer : embeddings[:5] = ', embeddings[:5])

embeddings = embedding_fn([text])
print ('milvus model wrapper : embeddings len =', len(embeddings[0]))
print ('milvus model wrapper  : embeddings[:5] = ', embeddings[0][:5])

sentence transformer : embeddings len = 384
sentence transformer : embeddings[:5] =  [ 0.02468892  0.10352131  0.0275264  -0.08551715 -0.01412829]
milvus model wrapper : embeddings len = 384
milvus model wrapper  : embeddings[:5] =  [ 0.02468898  0.10352132  0.02752642 -0.08551714 -0.01412826]


## Do A  Vector Search

We will do this to verify data

In [6]:
import random


## helper function to perform vector search
def  do_vector_search (query):
    query_vectors = [get_embeddings(query)]  # Option 1 - using sentence transformers
    # query_vectors = embedding_fn([query])  # using Milvus model 

    results = milvus_client.search(
        collection_name=MY_CONFIG.COLLECTION_NAME,  # target collection
        data=query_vectors,  # query vectors
        limit=5,  # number of returned entities
        output_fields=["filename", "page_number", "text"],  # specifies fields to be returned
    )
    return results
## ----

def  print_search_results (results):
    # pprint (results)
    print ('num results : ', len(results[0]))

    for i, r in enumerate (results[0]):
        #pprint(r, indent=4)
        print (f'------ result {i+1} --------')
        print ('search score:', r['distance'])
        print ('filename:', r['entity']['filename'])
        print ('page number:', r['entity']['page_number'])
        print ('text:\n', r['entity']['text'])
        print()

In [7]:
query = "What was Walmart's revenue in 2023?"

results = do_vector_search (query)
print_search_results(results)

num results :  5
------ result 1 --------
search score: 0.7401622533798218
filename: Walmart-10K-Reports-Optimized_2023.pdf
page number: 38
text:
 Strong, Efficient Growth
Comparable sales in the U.S., including fuel, increased 8.2% and 7.7% in fiscal 2023 and 2022, respectively, when compared to the previous fiscal year. Walmart U.S. comparable sales increased 7.0% and 6.4% in fiscal 2023 and 2022, respectively. For

------ result 2 --------
search score: 0.7293714284896851
filename: Walmart-10K-Reports-Optimized_2023.pdf
page number: 8
text:
 General
Our operations comprise three reportable segments: Walmart U.S., Walmart International and Sam's Club. Our fiscal year ends on January 31 for our United States ("U.S.") and Canadian operations. We consolidate all other operations generally using a one-month lag and on a calendar year basis. Our discussion is as of and for the fiscal years ended January 31, 2023 ("fiscal 2023"), January 31, 2022 ("fiscal 2022") and January 31, 2021 ("fisc

In [8]:
query = "How many distribution facilities does Walmart have?"

results = do_vector_search (query)
print_search_results(results)

num results :  5
------ result 1 --------
search score: 0.7357840538024902
filename: Walmart-10K-Reports-Optimized_2023.pdf
page number: 10
text:
 Walmart U.S. Segment
Distribution . We continue to invest in supply chain automation and utilize a total of 163 distribution facilities which are located strategically throughout the U.S. For fiscal 2023, the majority of Walmart U.S.'s purchases of store merchandise were shipped through these facilities, while most of the remaining store merchandise we purchased was shipped directly from suppliers. General merchandise and dry grocery merchandise is transported primarily through the segment's private truck fleet; however, we contract with common carriers to transport the majority of our perishable grocery merchandise. We ship merchandise purchased by customers on our eCommerce platforms by a number of methods from multiple locations including from our 34 dedicated eCommerce fulfillment centers, as well as leveraging our ability to ship or del

In [9]:
# milvus_client.close()